# Numerical integration of Friedmann eq dilution factor

This file comprehends the numerical integration of the Friedmann's ecuation for the decay epoch of a particle that dominates the energy budget of the universe before decaying and that decays whie being non-relativistic. We make the approximation that the SM d.o.f.  can be valued at $t = \tau_V$ in $\int^{\infty}_{x_0}g^{1/3}_{SM}(x)y(x)e^{-x}dx$ where $x = t\Gamma_V$. Also, we approximate the temperature of the universe as being the temperature of a radiation dominated universe through the particles' decay. That is necessary to value the SM d.o.f. that appear in the Friedmann's equation.

The dilution factor for the standard model comoving entropy takes the following form: $D_{SM} = \left( 1 +  \left[\frac{4}{3}\left(\frac{2\pi^2}{135}\right)^\frac{1}{3}\int^{\infty}_{x_0}y(x)e^{-x}dx \right] \, (g^{dec}_{SM})^\frac{1}{3}f^\frac{4}{3}_V\left(\frac{m_V^2}{\Gamma_V M_{pl}}\right)^\frac{2}{3}\right)^\frac{3}{4}$ 

Where $y(x)$ is defined as $y(x) \equiv x_0^{2/3}z(x)$ and $x_0 \equiv \Gamma_V\sqrt{3M_{pl}^2/\rho_{V0}}$ with $M_{pl}$ being the reduced Planck mass.

Our computation is performed to obtain the numerical factor $A = \frac{4}{3}\left(\frac{2\pi^2}{135}\right)^\frac{1}{3}\int^{\infty}_{x_0}y(x)e^{-x}dx$. In order to obtain it we have to solve the Friedmann equation in presence of the SM radiation and the mediator decaying into SM radiation. The equation is the following:

$\frac{z(x)'}{z(x)} = \frac{1}{x_0}\left(\frac{e^{-x}}{z(x)^3} + \frac{\rho_{r0}}{\rho_{V0}}\frac{1}{z(x)^4} + \frac{g^{dec\,1/3}_{SM}}{z(x)^4g_{SM}(x)^{1/3}}\int^{x}_{x_0}z(x')e^{-x'}dx'\right)^{1/2}$

where

$\rho_{r0} = \frac{\pi^2}{30}g_{SM}(x_0)T^4_0$

$\rho_{V0} = m_V f_V \left( \frac{2*\pi^2}{45} \right)g_{SM}(x_0) T^3_0$

On the other hand, $g_{SM}(x)$ are the degrees of freedom of the Standard Model at $x = \Gamma_V t$. $g^{dec}_{SM}$ is equal to $g_{SM}(x = 1)$ i.e. the Standard Model d.o.f. at the time of the decay $t = 1/\Gamma_V = \tau_V$.

As we are assuming Early Matter Domination during the decay epoch defined as $x_0 \lesssim x \lesssim 1$ (at $x = 1$ almost all particles have dacayed) then $\rho_{r0} \ll \rho_{V0}$ and the second term of the later equation can be thrown away. Using the above definitions we arrive to:

$\frac{y(x)'}{y(x)} = \left(\frac{e^{-x}}{y(x)^3} + \frac{g^{dec\,1/3}_{SM}}{y(x)^4g_{SM}(x)^{1/3}}\int^x_{x_0}y(x')e^{-x'}dx'\right)^{1/2}$

Now, to integrate numerically the later equation we need to define a new funcion to obtain a system of first order differential equations. We will call this new function $S(x)$ and define it as follows:

$S(x) \equiv \int^{x}_{x_0}y(x')e^{-x'}dx'$

so its derivative w.r.t. $x$ takes the form

$S(x)' = y(x) e^{-x}$

With the previous we arrive to the system of first order differential equations we are going to integrate numerically:

$\left\{  
\begin{array}{lcl}
y(x)' = \frac{1}{y(x)}\left(y(x)e^{-x} + \frac{1}{g(x,\tau_V)^{1/3}}S(x)\right)^{1/2} \\ 
S(x)' = y(x) e^{-x} 
\end{array}  
\right.$

In [1]:
import math as mt
import numpy as np
import scipy.integrate as sint

In [3]:
# First we define an array for different values of the particles' lifetime as our computation leaves this parameter free without doing the 
# g_SM(x=1) approximation in the Friedmann's equation.
tau_v = np.linspace(1e-11,0.03,int(1e2))    #defined in seconds

#We define the different functions and constants we will be using:

Mpl = 2.4e18#GeV #reduced Planck mass
T_0 = 100000000#GeV (100 PeV)
f_v = 0.02 #value for a relativistic decoupled dark photon that mediates between a dark sector and the observable one.

# SM d.o.f. function definition
def g_SM(T): #T in GeV
    gp = [106.75,96.25,95.25,92.25,86.25,75.75,72.25,61.75,17.25,15.25,14.25,10.75,7.409,3.909]
    if T > 173.3: g = gp[0]
    if T < 173.3 and T > 125.6: g = gp[1]
    if T < 125.6 and T > 91.2: g = gp[2]
    if T < 91.2 and T > 80.4: g = gp[3]
    if T < 80.4 and T > 4.19: g = gp[4]
    if T < 4.19 and T > 1.777: g = gp[5]
    if T < 1.777 and T > 1.29: g = gp[6]
    if T < 1.29 and T > 0.182: g = gp[7] 
    if T < 0.182 and T > 0.1396: g = gp[8]
    if T < 0.1396 and T > 0.135: g = gp[9]
    if T < 0.135 and T > 0.1057: g = gp[10]
    if T < 0.1057 and T > 0.0008: g = gp[11]
    if T < 0.0008 and T > 0.000511: g = gp[12]
    if T < 0.000511: g = gp[13]
    return g

def Trad(t): #t in seconds
    if t == 0: return 1e100
    else: return 0.001*np.sqrt(2.42/t)/g_SM(0.001*np.sqrt(2.42/t))**(1/4)


#SOLVE FRIEDMANN EQUATION FOR x0 APPROX 0 AND COMPUTE THE VALUE OF
# THE NUMERICAL FACTOR IN THE DILUTION FACTOR EXPRESSION:
def g(x,tau):
    return g_SM(Trad(tau*x))/g_SM(Trad(tau))
x_0 = 1e-3
u0_0 = x_0**(2/3)
u0_1 = 0
u0 = [u0_0,u0_1]
X = np.linspace(x_0,10,int(1e6))
rtol, atol = (1e-8, 1e-8)
I = np.zeros([1,len(tau_v)])
i = 0
for tau in tau_v:
    def F(x,u):
        return [ np.sqrt(u[0]*np.exp(-x) + u[1]*g(x,tau)**(-1/3))/u[0] , u[0]*np.exp(-x) ]
    sol1 = sint.solve_ivp(F,[x_0, 10],u0,rtol = rtol, atol = atol, t_eval = X)
    S = sol1.y[1]
    I[0][i] = S[-1]
    i+=1





Our result is in the next cell (execute the next cell):

In [7]:
# print(np.mean(I))
# print(np.mean(I)*4/3*(8*mt.pi/3)**(1/3))
A = 1.09*4/3*(2*mt.pi**2/135)**(1/3)
print(A)

0.7656504793127655


In [1]:
# x_0 = 1e-3
# tau = 1e-11
# u0_0 = x_0**(2/3)
# u0_1 = 0
# u0 = [u0_0,u0_1]
# X = np.linspace(x_0,10,int(1e6))
# rtol, atol = (1e-8, 1e-8)
# # for i in X:
# #     print(g_SM(Trad(tau*i))/g_SM(Trad(tau)))
# # u(x) = (y(x),S(x))
# def F(x,u):
#     return [ np.sqrt(u[0]*np.exp(-x) + u[1])/u[0] , u[0]*np.exp(-x) ]
# sol1 = sint.solve_ivp(F,[x_0, 10],u0,rtol = rtol, atol = atol, t_eval = X)
# S = sol1.y[1]
# I = S[-1]
# print(I*4/3*(2*mt.pi**2/135)**(1/3))